# Cross Validation

## The holdout method
 - 데이터가 적을 경우 사용 => 데이터가 많다면 운에 맡겨서 나온 것이라고 봐도 무방하다.
 - 현재 수업 중에 테스트하는 6:4 비율 활용이 이에 해당한다. 
 <img src="photo/12.PNG">

## K-fold Cross Validation(KFCV)
<img src="photo/13.PNG">

In [13]:
library(caret)
library(randomForest)
library(ROCR)

In [4]:
cb <- read.delim("../1022_Decision Tree_2/Hshopping.txt", stringsAsFactors=FALSE)
colnames(cb) <- c("ID","SEX","AGE","AMT","STAR","REFUND") # Jupyter note Font Error using Korean
cb$REFUND <- factor(cb$REFUND)

set.seed(1)
flds <- createFolds(cb$REFUND, k=5, list=T, returnTrain=F)

In [6]:
str(flds)

List of 5
 $ Fold1: int [1:99] 5 7 12 14 18 19 21 25 30 32 ...
 $ Fold2: int [1:101] 11 33 39 41 45 80 85 86 90 94 ...
 $ Fold3: int [1:100] 10 15 16 22 24 31 34 35 36 48 ...
 $ Fold4: int [1:100] 2 3 4 8 9 13 20 23 26 29 ...
 $ Fold5: int [1:100] 1 6 17 27 28 43 46 47 53 54 ...


### Perform 5 experiments

In [7]:
experiment <- function(train, test, m) {
    rf <- randomForest(REFUND ~ .-ID, data=train, ntree=50)
    rf_pred <- predict(rf, test, type="response")
    m$acc = c(m$acc, confusionMatrix(rf_pred, test$REFUND)$overall[1])
    rf_pred_prob <- predict(rf, test, type="prob")
    rf_pred <- prediction(rf_pred_prob[,2], cb.test$REFUND)
    m$auc = c(m$auc, performance(rf_pred, "auc")@y.values[[1]])
    return(m)
}

In [14]:
measure = list()
for(i in 1:5){
    inTest <- flds[[i]]
    cb.test <- cb[inTest,]
    cb.train <- cb[-inTest,]
    
    measure = experiment(cb.train,cb.test,measure)
}

In [16]:
measure

$acc
 Accuracy  Accuracy  Accuracy  Accuracy  Accuracy 
0.8888889 0.8712871 0.9200000 0.9100000 0.9100000 

$auc
[1] 0.9670304 0.9576540 0.9607293 0.9588593 0.9815334

In [17]:
mean(measure$acc); sd(measure$acc)

[1] 0.9000352

[1] 0.01967155

In [18]:
mean(measure$auc); sd(measure$auc)

[1] 0.9651613

[1] 0.009839432